# Tutorial on Helpers for Model Fitting

This tutorial discusses helper codes you can use for model fitting with tellurium. The tutorial gives practical examples of to use these codes in a model fitting workflow.

The utilities are in 3 python modules (files):

- named_timeseries defines a NamedTimeseries type that allows you to manipulate tabular data that has a time axis. A NamedTimeseries can be initialized from a comma separated variable (CSV) file.
- model_fitter defines a ModelFitter type that encapsulates the basics of fitting parameters to a NamedTimeseries. You can simulate the fitted model, and the result is a NamedTimeseries.
- timeseries_plotter plots one or two NamedTimeseries. You can have all variables on a single subplot or have separate subplots for each variable.

You should open this notebook from ``teUtils``, then open the folder ``notebooks`` to find this notebook.

## Quick Start

In [1]:
# Basic imports
from teUtils.model_fitter import ModelFitter 

In [2]:
# Model of a linear pathway
ANTIMONY_MODEL = """ 
# Reactions   
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S4; k3*S3
    J4: S4 -> S5; k4*S4
    J5: S5 -> S6; k5*S5;
# Species initializations
    S1 = 10;
    S2 = 0;
    S3 = 0;
    S4 = 0;
    S5 = 0;
    S6 = 0;
    k1 = 1;
    k2 = 2;
    k3 = 3;
    k4 = 4;
    k5 = 5;
"""

In [3]:
# Fit parameters to ts1
fitter = ModelFitter(ANTIMONY_MODEL, "tst_data.txt", ["k1", "k2", "k3", "k4", "k5"])
fitter.fitModel()

In [ ]:
# Provide fit details
print(fitter.reportFit())

In [ ]:
# Plot the fitl
fitter.plotFitAll(num_row=2, num_col=3)

In [ ]:
# Look at the plotting options
fitter.plotFitAll(is_help=True)

In [ ]:
# Plot residuals
fitter.plotResiduals(num_row=2, num_col=3)

In [ ]:
# Get estimates of parameter variance by using bootstrapping
fitter.bootstrap(num_iteration=20, report_interval=5)
print(fitter.getFittedParameterStds())

## Detailed Tutorial

In [ ]:
# Import the modules needed.
import numpy as np
import tellurium as te

from teUtils.named_timeseries import NamedTimeseries, TIME
from teUtils.timeseries_plotter import TimeseriesPlotter, PlotOptions
from teUtils.model_fitter import ModelFitter


## Manipulating Tabular Data
Tables are a great way to describe data. Tables are a collection of values organized into rows and columns. Typically, the columns have names. Common operations on tables are:

- Create a table from a file
- Create a new table from a subset of rows and/or columns
- Create new columns and/rows by performing addition, subtaction, and other operations on table rows and columns.

Many systems provide table representations. Spreadsheets are widely used. In the R language, a dataframe is a table. In python, the pandas package provides a python version of dataframes.

Pandas is very powerful, but it is slow. numpy provides arrays and matrices, but these lack the convenience of column names. In addition, the variables we analyze are collected at the same timepoints, and so it's important to associate these timepoints with the variable values.

``NamedTimeseries`` is a thin layer on top of numpy ``ndarray`` that provides an efficient table abstraction. Efficiency is a particular concern for model fitting since fitting can be very numerically intensive.

Let's start by creating a ``NamedTimeseries`` from the file ``tst_data.txt``. In python, an object is created from a constructor that is the class name followed by arguments. There are several constructors for ``NamedTimeseries``.

In [ ]:
ts1 = NamedTimeseries(csv_path="tst_data.txt")  # Construct from a file
print(ts1)

There are 7 columns (including "time") and 30 rows. All ``NamedTimeseries`` have a "time" column. In fact, the python variable ``TIME`` can be used to reference this column.

We reference rows by their "index", their count from 0. In fact, we can make a new ``NamedTimeseries`` that consists of a subset of the rows.

In [ ]:
# Get the first row
print(ts1[1])

In [ ]:
# Get rows 1, 3, 5
print(ts1[[1, 3, 5]])

In [ ]:
# Get rows 1 through 5
# This is a slice; you specify the first row you want and where the slice stops
ts2 = ts1[1:6]
print(ts2)

We can also select columns. A column select produces a ``numpy`` ``ndarray``, not at ``NamedArray``. This is done so that we can manipulate column selections using the powerful numpy facilities.

In [ ]:
ts2["S1"]  # Column S1 in ts2

To get the numpy array for a list of columns, we give the list.

In [ ]:
ts2[["S1", "S2"]]

To get a **new timeseries** that just has "S1", you use ``subsetColumns``.

In [ ]:
ts2a = ts2.subsetColumns("S1")
print(ts2a)

In [ ]:
# Create a timeseries from ts2 with just S1, S2
print(ts2.subsetColumns(["S1", "S2"]))

We can add columns to the timeseries. Here, we add a column with a constant value of 1.

In [ ]:
ts2["constant_1"] = 1
print(ts2)

We can also do calculations and create new columns with the results.

In [ ]:
ts2["New_Calculation"] = ts2["S1"] + np.multiply(ts2["S2"], ts2[TIME])
ts2["Other_Calculation"] = ts2["S1"] + 10*np.multiply(ts2["S2"], ts2[TIME])
print(ts2)

In [ ]:
# Let's get rid of Other_Calculation
del ts2["Other_Calculation"]
print(ts2)

We can combine the rows and columns of a ``NamedTimeseries`` to make a new ``NamedTimeseries``.

In [ ]:
ts3 = ts2.subsetColumns(["S1", "S2"])
ts4 = ts2.subsetColumns("S3")
ts5 = ts2.subsetColumns("New_Calculation")
ts6 = ts3.concatenateColumns([ts4, ts5])  # Has S1, S2, S3, New_Calculation"
print(ts6)

In [ ]:
ts7 = ts1[:2]
ts8 = ts1[2:4]
ts9 = ts1[4:6]
ts10 = ts7.concatenateRows([ts8, ts9])  # First 6 rows of ts1
print(ts10)

In [ ]:
# Save the result in a file
ts2.to_csv("new_data.csv")

In [ ]:
# To see the file, we run a command in the operating system shell. The following works in Unix and Mac.
!cat new_data.csv

## Plotting Tabular Data
We provide 4 types of plots with user controllable options for the plots. The plots are:

1. Plot all variables of one table on a single plot.
1. Plot all variables of one table in one plot and all variables of second plot
1. Plot each variable of a table in a separate plot
1.  Plot each variable of two tables in the same plot

In [ ]:
# Create the plotter object
plotter = TimeseriesPlotter()

In [ ]:
# To see the plotting options...
print(PlotOptions())

In [ ]:
# See the options for plotting
help(PlotOptions)

In [ ]:
# Case 1: Plot all variables of a single table on a single plot
plotter.plotTimeMultiple(ts1)
plotter.plotTimeMultiple(ts1, timeseries2=ts1, marker2="o")
plotter.plotTimeMultiple(ts1, timeseries2=ts1, marker2="o", num_col=1, num_row=1)

In [ ]:
# We can also plot a subset of the variables in a single plot
plotter.plotTimeMultiple(ts1, columns=["S1", "S6"])

In [ ]:
# Add to the plot with options
plotter.plotTimeMultiple(ts1, columns=["S1", "S6"], ylabel="concetration", title="Two Variables")

In [ ]:
# Create a second table for comparisons. We can access all non-time variables using the property colnames.
ts4 = ts1.copy()  # Create a copy of the timeseries
ts4[ts4.colnames] = 5*np.cos(ts4[ts4.colnames]) + 2*ts4[ts4.colnames]

In [ ]:
# Case 2: Plot two multiple variable plots, side-by-side
plotter.plotTimeMultiple(ts1, timeseries2=ts4)

In [ ]:
# Case 3: Separate plot for each variable of a single table
plotter.plotTimeSingle(ts1, num_row=2, num_col=3, timeseries2=ts2,
                      figsize=(10,8), marker1="o")

In [ ]:
# Case 4: Separate plot for each variable for two tables
plotter.plotTimeSingle(ts1, timeseries2=ts2, figsize=(10, 8))

## Fitting Parameters of a Model
Kinetics models frequently have unknown constants that need to be estimated. This process of estimation is called model fitting, or just fitting. Fitting involves running multiple simulations with different values of parameters to find values that result in a good match between simulated and measured species concentrations.

In [ ]:
ANTIMONY_MODEL = """ 
# Reactions   
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S4; k3*S3
    J4: S4 -> S5; k4*S4
    J5: S5 -> S6; k5*S5;
# Species initializations
    S1 = 10;
    S2 = 0;
    S3 = 0;
    S4 = 0;
    S5 = 0;
    S6 = 0;
    k1 = 1;
    k2 = 2;
    k3 = 3;
    k4 = 4;
    k5 = 5;
"""

This model has the parameters ``k1, k2, k3, k4, k5``, the kinetics constants for their respective reactions. In this case, we know the true value of the parameters, which are specified at the bottom of the model. This knowledge provides a way to explore the effectiveness of model fitting.

This model is a linear pathway where mass moves from species ``S1`` to ``S6`` over time. The rate at which this happens depends on the kinetics constants.

Let's see how well we can fit this model to the ``ts1`` data described in the first section.

In [ ]:
print(ts1)

The fitting data has values for each of the six floating species. Model fitting works iteratively varying the values of the parameters, simulating the model with these parameters, and then evaluating how close the simulation is to the fitting data.

So, to do a fit, we must specify:

1. The model to fit
1. The fitting data
1. The parameters to estimate

To elaborate on point (3), often models contain some parameters whose values are known and others that aren't.

The class ``ModelFitter`` is used to do fitting. The constructor must specify all 3 items above. If parameters is None, then no fitting is done.

In [ ]:
# Run the model using the parameter values in the model. There is no fitting.
base_fitter = ModelFitter(ANTIMONY_MODEL, ts1, None)

To fit a model, we use the ``fitModel`` method. We can then see the optimized residual variance and the values of the model parameters.

In [ ]:
base_fitter.fitModel()

In [ ]:
# The model without fitting any parameters
ts_base = NamedTimeseries(named_array=base_fitter.roadrunner_model.simulate(0, 4, 30))
plotter.plotTimeSingle(ts_base, num_row=2)

This is a bit crowded for detailed comparisons. So, we make comparisons between each variable.

In our running example, the values of parameters in the original model are their true value. Ideally, we want to achieve a variance close to this. We create a new fitter object since we still want to have access to the unchanged model that we started with.

In [ ]:
parameter_names = ["k%d" % i for i in range(1, 6)]
fitter = ModelFitter(ANTIMONY_MODEL, ts1, parameter_names)
fitter.fitModel()
print("residual variance of the fitted model: %3.2f" % np.var(fitter.residuals_ts.flatten()))
print("Values of fitted parameters: %s" % str(fitter.getFittedParameters()))

Now let's compare the fitted model with the data.

In [ ]:
fitted_model = fitter.roadrunner_model
fitted_model.reset()
fitted_ts = NamedTimeseries(named_array=fitted_model.simulate(ts1.start, ts1.end, len(ts1)))
plotter.plotTimeMultiple(ts1, timeseries2=fitted_ts, marker1="o")

In [ ]:
plotter.plotTimeSingle(ts1, timeseries2=fitted_ts, num_row=2, num_col=3, figsize=(12, 10), marker1="o")

In [ ]:
# Plot pairs of values
plotter.plotValuePairs(fitted_ts, pairs=[("S1", "S2"), ("S1", "S6"), ("S2", "S3")], num_col=3)

In [ ]:
fitter.plotFitAll(num_col=1, num_row=1, is_multiple=True)

In [ ]:
fitter.plotFitAll(num_row=2, num_col=2, figsize=(12, 10))